# Actividad 2:
# Aplicación comparativa de técnicas avanzadas de regresión

## Objetivo
Preprocesar y analizar datos reales aplicando técnicas avanzadas de regresión, evaluando modelos desde una perspectiva técnica y contextual, y comparando enfoques para seleccionar el más adecuado según el problema.

**Datasets utilizados:**  
`California Housing`
`Adult Income Dataset`
`Macroeconomic Dataset`

---

### Estructura del Notebook:
1. Metodología.
2. Importación de librerias a utilizar.
3. Definicion de funciones.
4. Uso de funciones y resultados.
5. Análisis de los resultados y reflexiones finales.

---

## 1. Metodología

### Flujo de trabajo

1. **Carga y preprocesamiento de datos y entrenamiento de modelos:**

Se utilizaron 3 datasets para evaluar 3 técnicas avanzadas de regresión:  

   - Se utilizó el dataset **California Housing** desde `fetch_california_housing` para evaluar Elastic Net.
   - Se utilizó el dataset **Adult Income** desde `fetch_openml` para evaluar regresión cuantílica (0.1, 0.5 y 0.9).
   - Se utilizó el dataset **Macroeconomic** desde `load_pandas` para evaluar VAR.

Para cada caso se definió una función en donde se cargaba el dataset, se preprocesaban y escalaban los datos, y finalmente se obtenian las métricas. 

2. **Evaluación y análisis:**
   - Métricas empleadas:
     - **RMSE:** se utilizó como métrica para evaluar Elastic Net.
     - **Pinball Loss:** para evaluar Regresión Cuantílica.
     - **Visualización de series y errores:** como método de evaluación de VAR.

3. **Visualización de resultados:**
  - Tabla resumen con métricas evaluadas para cada modelo.
  - Gráficos de barras comparativos para mostrar coeficientes mas importantes en Elastic Net y Regresión Cuantílica (0.1, 0.5 y 0.9).
  - Gráfico de pronóstico de los proximos 5 pasos para **realgdp**, **realcons** y ***realinv**.

---

# 2. Importacion de librerias necesarias

--- 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing, fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import root_mean_squared_error, mean_pinball_loss
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.datasets.macrodata import load_pandas
import optuna
import warnings

warnings.filterwarnings("ignore")

# Configuración visual y de entorno
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.precision', 4)
pd.set_option('display.max_columns', None)

# 3. Definición de funciones

> **Nota:** Para mejor comprensión de las funciones y su utilidad, esta sección se divide en bloques, en donde cada uno responde a una parte diferente de la metodología de trabajo. 

---

**Bloque 1:** Carga y preprocesamiento de datos y entrenamiento de modelos.

- **`entrenar_elasticnet_california()`** 
Entrena un modelo ElasticNet en el dataset California Housing y devuelve su RMSE y las 5 variables más importantes. Se usa Optuna para elegir los mejores hiperparametros (alpha y l1_ratio).

- **`entrenar_quantileregression_adult()`** 
Ajusta regresión cuantilica en el dataset Adult Income y calcula la pérdida Pinball y coeficientes por cuantil.

- **`entrenar_var_macro()`** 
Ajusta un modelo VAR sobre datos macroeconómicos y predice las próximas 5 observaciones.

In [ ]:
def entrenar_elasticnet_california():
    """
    Entrena un modelo ElasticNet optimizado con Optuna sobre el dataset California Housing.

    Se realiza búsqueda de hiperparámetros (alpha, l1_ratio) para minimizar el RMSE.
    Luego se entrena el modelo final con los mejores parámetros y se obtienen las 5 variables más importantes.

    Returns:
        float: RMSE del modelo final sobre el conjunto de prueba.
        pd.Series: Serie con las 5 variables más importantes ordenadas por su peso absoluto.
    """
    print("[INFO] Entrenando Elastic Net con Optuna en California Housing...")

    # Cargar y dividir el dataset
    california = fetch_california_housing(as_frame=True)
    X, y = california.data, california.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalado
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Función objetivo para Optuna
    def objective(trial):
        alpha = trial.suggest_float("alpha", 1e-4, 1.0, log=True)
        l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        scores = cross_val_score(model, X_train_scaled, y_train, scoring='neg_root_mean_squared_error', cv=5)
        return -np.mean(scores)

    # Ejecutar optimización
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50, show_progress_bar=False)

    best_alpha = study.best_params['alpha']
    best_l1_ratio = study.best_params['l1_ratio']
    print(f"[INFO] Mejores parámetros: alpha={best_alpha:.5f}, l1_ratio={best_l1_ratio:.3f}")

    # Entrenar modelo final
    model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    rmse = root_mean_squared_error(y_test, y_pred)

    # Importancia de variables
    importance = pd.Series(model.coef_, index=X.columns)
    top_features = importance.abs().sort_values(ascending=False).head(5)

    return rmse, top_features

def entrenar_quantileregression_adult():
    """
    Entrena modelos de regresión cuantilica (Quantile Regression) sobre el dataset Adult Income.

    Para cada cuantil (0.1, 0.5, 0.9), entrena un modelo usando statsmodels y evalúa su rendimiento
    con la pérdida Pinball. También devuelve los coeficientes del modelo para cada cuantil.

    Returns:
        dict: Diccionario con los cuantiles como llaves y sus respectivas pérdidas Pinball como valores.
        dict: Diccionario con los cuantiles como llaves y series de coeficientes del modelo como valores.
    """
    print("[INFO] Entrenando Regresión Cuantílica con Adult Income...")
    adult = fetch_openml("adult", version=2, as_frame=True)
    df = adult.frame.copy().replace('?', np.nan).dropna()
    df['target'] = df['class'].apply(lambda x: 1 if x == '>50K' else 0)

    X = pd.get_dummies(df.drop(columns=['class', 'target']), drop_first=True)
    y = df['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    X_train_const = sm.add_constant(X_train_df, has_constant='add')
    X_test_const = sm.add_constant(X_test_df, has_constant='add')
    X_test_const = X_test_const[X_train_const.columns]

    pinball_losses = {}
    coef_dict = {}
    for q in [0.1, 0.5, 0.9]:
        print(f"  > Entrenando QuantileReg q={q} con statsmodels...")
        qr = sm.QuantReg(y_train, X_train_const)
        res = qr.fit(q=q)
        y_pred = res.predict(X_test_const)
        pinball_losses[q] = mean_pinball_loss(y_test, y_pred, alpha=q)
        coef_dict[q] = pd.Series(res.params, index=X_train_const.columns)

    return pinball_losses, coef_dict

def entrenar_var_macro():
    """
    Entrena un modelo VAR (Vector Autoregressive) sobre datos macroeconómicos (realgdp, realcons, realinv).

    Se aplica diferenciación para garantizar estacionariedad, se selecciona el mejor rezago según AIC,
    y se realiza un pronóstico de los próximos 5 pasos.

    Returns:
        int: Número de rezagos óptimo seleccionado por criterio AIC.
        pd.DataFrame: DataFrame con el pronóstico de las tres variables para los próximos 5 pasos.
    """
    print("[INFO] Entrenando modelo VAR con datos macroeconómicos...")
    macro = sm.datasets.macrodata.load_pandas().data
    macro.index = pd.date_range(start="1959Q1", periods=len(macro), freq='Q')
    df = macro[["realgdp", "realcons", "realinv"]].copy()
    df_diff = df.diff().dropna()

    model = VAR(df_diff)
    lag_order = model.select_order(10).selected_orders['aic']
    var_model = model.fit(lag_order)
    forecast = var_model.forecast(df_diff.values[-lag_order:], steps=5)
    forecast_df = pd.DataFrame(forecast, columns=df_diff.columns, index=[f"Paso {i+1}" for i in range(5)])
    return lag_order, forecast_df

**Bloque 3:** Resultados y visualizaciones.

- **`visualizar_resultados_elasticnet()`** 
Grafica las variables más influyentes del modelo ElasticNet.

- **`visualizar_coeficientes_cuantilicos()`** 
Muestra los coeficientes relevantes de cada modelo cuantilico en gráficos de barras.

- **`visualizar_forecast_var()`** 
Grafica el pronóstico de variables macroeconómicas producido por el modelo VAR.

In [ ]:
def visualizar_resultados_elasticnet(top_features):
    """
    Genera un gráfico de barras con las variables más importantes según el modelo ElasticNet.

    Args:
        top_features (pd.Series): Serie con las variables más relevantes y sus coeficientes absolutos.
    """
    plt.figure(figsize=(10, 5))
    top_features.plot(kind='bar')
    plt.title("Elastic Net - Coeficientes más importantes")
    plt.ylabel("Peso")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig("elastic_net_top_features.png")
    plt.show()

def visualizar_coeficientes_cuantilicos(coef_dict):
    """
    Visualiza los coeficientes más significativos para cada cuantil en modelos de regresión cuantilica.

    Para los cuantiles 0.1, 0.5 y 0.9, se muestran los 10 coeficientes más importantes en gráficos de barras,
    si es que superan un umbral de 0.1 en valor absoluto.

    Args:
        coef_dict (dict): Diccionario con cuantiles como llaves y Series de coeficientes como valores.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    for ax, q in zip(axes, [0.1, 0.5, 0.9]):
        coef = coef_dict[q].copy()
        top = coef[coef.abs() > 0.1].sort_values(key=np.abs, ascending=False)[:10]
        if not top.empty:
            top.plot(kind='bar', ax=ax)
            ax.set_title(f"Quantile {q} - Top coeficientes")
            ax.set_ylabel("Peso")
            ax.set_xticklabels(top.index, rotation=45, ha='right')
        else:
            ax.set_title(f"Quantile {q} - Sin coeficientes significativos")
    plt.tight_layout()
    plt.savefig("regrecion_cuantilica_top_features.png")
    plt.show()

def visualizar_forecast_var(forecast_df):
    """
    Muestra el pronóstico generado por el modelo VAR para las próximas 5 observaciones.

    Se grafican las series de tiempo proyectadas para las variables macroeconómicas.

    Args:
        forecast_df (pd.DataFrame): DataFrame con el pronóstico de las variables macroeconómicas.
    """
    forecast_df.plot(marker='o', figsize=(10, 6))
    plt.title("VAR - Pronóstico próximos 5 pasos (diferencias trimestrales)")
    plt.xlabel("Paso")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("var_pronostico.png")
    plt.show()

**Bloque 3:** Función de ejecución.

- **`main()`**
Ejecuta todo el flujo de entrenamiento, evaluación y visualización para los tres modelos.

In [ ]:
def main():
    """
    Ejecuta el flujo completo del análisis de tres modelos distintos:

    - Entrena un modelo ElasticNet con el dataset California Housing y muestra su RMSE.
    - Entrena modelos de regresión cuantilica con el dataset Adult Income y muestra la pérdida Pinball.
    - Ajusta un modelo VAR sobre datos macroeconómicos y muestra el pronóstico para los próximos 5 pasos.
    - Visualiza las variables más importantes de cada modelo mediante gráficos.

    No recibe ni retorna argumentos; imprime resultados en consola y genera visualizaciones.
    """
    rmse, top_features = entrenar_elasticnet_california()
    pinball_losses, coef_dict = entrenar_quantileregression_adult()
    lag_order, forecast_df = entrenar_var_macro()

    print("\n[RESUMEN FINAL DE MODELOS]")
    print(f"Elastic Net RMSE (California Housing): {rmse:.4f}\n")

    print("Regresión Cuantílica (Pinball Loss - Adult Income):")
    for q, loss in pinball_losses.items():
        print(f"  Quantile {q}: {loss:.4f}")
    print()

    print(f"VAR - Lag óptimo seleccionado: {lag_order}")
    print("Pronóstico de los próximos 5 pasos:")
    print(forecast_df)

    visualizar_resultados_elasticnet(top_features)
    visualizar_coeficientes_cuantilicos(coef_dict)
    visualizar_forecast_var(forecast_df)

# 4. Visualización de resultados

Se muestran los resultados obtenidos a partir de la ejecución de la funcion **main()**.

---

In [ ]:
if __name__ == "__main__":
    main()

# 5. Análisis de los resultados y relfexiones finales

>**NOTA**: Los resultados aca expuestos y analizados fueron sacados de una ejecución x, por lo que, al momento de ejecutar nuevamente el código los resultados podrían variar, principalmente el de Elastic Net, debido a que Optuna podría no encontrar la misma combinación de hiperparametros que en esta ejecución. Debido a lo anterior, se recomienda tomar los valores analisados como aproximaciones y no como valores exactos, aunque las variaciones no deberían ser tan significativas.

## Elastic Net (Regresión - California Housing)

- **Optimización**: Se utilizó Optuna para buscar los mejores hiperparámetros (alpha, l1_ratio) mediante validación cruzada y minimización del RMSE.Este enfoque permitió un ajuste más fino y mejor desempeño del modelo.

- **Rendimiento**: RMSE original sin optimizar fue de **0.7974**, mientras que después de optimizar, el RMSE disminuyo a **0.7205**, lo que indica que la optimización encontro mejores valores para alpha y l1_ratio que los originales (0.1 y0.5 respectivamente), lo cual es aceptable para un modelo lineal simple.

- **Variable más influyente**: Tras la optimización con Optuna, `Latitude` y `Longitude` pasaron a ser las variables con mayor peso, superando a `MedInc` (ingreso medio). Esto se debe a que la regularización fuerte favoreció variables espaciales que capturan la variabilidad del precio relacionada con la ubicación, una señal muy relevante en el mercado inmobiliario.

- **Ventajas**:
  - Técnica de regularización combinada (L1 + L2) que previene sobreajuste.
  - Modelo interpretable y computacionalmente eficiente.

- **Limitaciones**:
  - Al ser un modelo lineal, **no capta relaciones no lineales complejas**.
  - Coeficientes sensibles a multicolinealidad entre variables.

---

## Regresión Cuantílica (Clasificación - Adult Income)

- **Optimización**: No se aplicó búsqueda de hiperparámetros, ya que el modelo en statsmodels no expone parámetros como alpha o max_iter. Cada modelo se entrena para un cuantil distinto (q) y se evalúa individualmente.

- **Pinball Loss**:
  - **q = 0.1** → 0.0245 (muy bajo).
  - **q = 0.5** → 0.1128.
  - **q = 0.9** → 0.0537.
- **Variables más influyentes**:
  - **q = 0.1**: Sin coeficientes relevantes detectados.
  - **q = 0.5**: `education_Masters`.
  - **q = 0.9**: `const`, `marital-status`.

**Análisis de Coeficientes:**

- En **q=0.1**, no se observaron coeficientes relevantes, lo que indica que los ingresos más bajos son difíciles de predecir con las variables disponibles, posiblemente por factores estructurales o no observados.
- En **q=0.5** (mediana), destaca `education_Masters`, lo cual es consistente con la evidencia de que tener estudios de posgrado está asociado a mayores ingresos.
- En **q=0.9**, las variables `marital-status` y el intercepto (`const`) son dominantes. Esto sugiere que:
  - **Estado civil** es un fuerte diferenciador en los percentiles más altos de ingreso, posiblemente por su correlación con estabilidad laboral, edad o composición familiar.
  - El valor alto de `const` indica que incluso sin otras variables predictoras, hay un componente estructural que empuja los ingresos altos en ese cuantil.

- **Ventajas**:
  - Permite explorar cómo distintas variables afectan diferentes percentiles del ingreso.
  - Útil en estudios de desigualdad o análisis económico más granular.

- **Limitaciones**:
  - **No es un modelo óptimo para clasificación binaria**.
  - Algunos cuantiles no produjeron coeficientes significativos.
  - Puede ser computacionalmente costoso y sensible al preprocesamiento.

---

## VAR (Series Temporales - Macroeconomic Data)

- **Optimización**: No se utilizó Optuna u otra técnica de optimización, ya que statsmodels para VAR ya incluye selección automática de rezagos usando criterios como AIC. En este contexto, usar búsqueda bayesiana o aleatoria no aporta beneficios significativos.

- **Lag óptimo**: 3 (según criterio AIC).

- **Pronóstico**:
  - `realgdp` y `realcons` muestran tendencia creciente.
  - `realinv` se contrae en los primeros pasos del pronóstico.

- **Ventajas**:
  - Modelo multivariado que captura la dinámica conjunta de varias series económicas.
  - Útil para pronósticos de corto plazo y análisis de shock.

- **Limitaciones**:
  - Requiere **estacionariedad previa** (se aplicó diferenciación).
  - Interpretación más compleja sin análisis adicional (e.g., funciones impulso-respuesta).

---

### Conclusión Comparativa

| Técnica                 | Más útil para                            | Limitaciones principales                             |
|-------------------------|------------------------------------------|------------------------------------------------------|
| **Elastic Net**         | Regresión sobre variables numéricas      | No capta relaciones no lineales complejas            |
| **Regresión Cuantílica**| Análisis en distintos niveles de ingreso | No está optimizada para clasificación                |
| **VAR**                 | Series económicas multivariadas          | Requiere diferenciación e interpretación cuidadosa   |